In [1]:
# Importing needed library
import pandas as pd
import numpy as np
import beta_glu
import pandascharm as pc
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

from sklearn.preprocessing import OneHotEncoder
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import blosum as bl
from Bio.SubsMat.MatrixInfo import blosum62 as blosum
from Bio import AlignIO
from Bio import SeqIO
from Bio.Align.Applications import MuscleCommandline
from Bio.Align import AlignInfo
from scipy.fft import fft, ifft

from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from pyaaisc import Aaindex
from Bio import AlignIO


/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [2]:
df = pd.read_excel('../beta-glucosidase/data/Review Paper Data Final.xlsx')

In [3]:
output = 'pNP-Glc Ki (mM)'
encoding = 'BLOSUM62'
aln = '../beta-glucosidase/data/kcatKm w_Temperatures.aln'

In [4]:
X,y,holder = beta_glu.encode(encoding, output, df, aln, key = None)

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=101)
all_result = []

In [6]:
# DT Tuning
def DT(X_train, y_train, X_val, jack = False):
    knife = len(X_train)-1
    param_grid={ "max_features": [2,4,6,8,10,12], 'random_state':[20]}
    tree_reg=DecisionTreeRegressor()
    if jack == False:
        grid_search=GridSearchCV(tree_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(tree_reg, param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train,np.ravel(y_train))
    DT_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    param = grid_search.best_params_
    return DT_pred, pred_training, param

# SVM Tuning
def SVM(X_train, y_train, X_val, jack = False):
    knife = len(X_train)-1
    param_grid = {'C' : [1,5,10],'degree' : [3,8],'coef0' : [0.01,10,0.5]}
 
    model = SVR()
    if jack == False:
        grid_search=GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(model, param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train, y_train)
    SVM_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    param = grid_search.best_params_
    return SVM_pred, pred_training, param

# Random Forest Tuning
def RF(X_train, y_train, X_val, jack = False):
    knife = len(X_train)-1
    
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 50, stop = 150, num = 10)]
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Create the random grid
    param_grid = {'n_estimators': n_estimators,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf}
    rf = RandomForestRegressor()
    if jack == False:
        grid_search=GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(rf, param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train,np.ravel(y_train))
    RF_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    param = grid_search.best_params_
    return RF_pred, pred_training, param


# XGBR tuning
def XGBR(X_train, y_train, X_val, jack = False):
    knife = len(X_train)-1
    
    param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
    
    if jack == False:
        grid_search=GridSearchCV(xgb.XGBRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(xgb.XGBRegressor(), param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train, y_train)
    XGB_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    param = grid_search.best_params_
    return XGB_pred, pred_training, param

# NNR Tuning
def NNR(X_train, y_train, X_val, jack = False):
    
    knife = len(X_train)-1
    param_grid = {"hidden_layer_sizes": [1,50], "activation": ["identity", "logistic", "tanh", "relu"], "solver": ["lbfgs", "sgd", "adam"], "alpha": [0.00005,0.0005]}
    regr = MLPRegressor(random_state=101, max_iter=100).fit(X_train, y_train)
    if jack == False:
        grid_search=GridSearchCV(regr, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(regr, param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train, y_train)
    NNR_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    param = grid_search.best_params_
    return NNR_pred, pred_training, param

def EN(X_train, y_train, X_val, jack = False):
    knife = len(X_train)-1
    param_grid = {"max_iter": [1, 5, 10], "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],"l1_ratio": np.arange(0.0, 1.0, 0.1)}
    ENmodel = ElasticNet()
    if jack == False:
        grid_search=GridSearchCV(ENmodel, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(ENmodel, param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train, y_train)
    EN_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    param = grid_search.best_params_
    return EN_pred, pred_training,param


In [7]:
DT_pred , DT_train, param = DT(X_train, y_train, X_val)
all_result.append([output, "Decision Tree Regression", encoding,holder, np.sqrt(metrics.mean_squared_error(y_train, DT_train)), np.sqrt(metrics.mean_squared_error(y_val, DT_pred)), r2_score(y_train, DT_train),r2_score(y_val, DT_pred), str(param)]) 

In [8]:
SVM_pred , SVM_train, param = SVM(X_train, y_train, X_val)
all_result.append([output, "Support Vector Machine Regression", encoding,holder, np.sqrt(metrics.mean_squared_error(y_train, SVM_train)), np.sqrt(metrics.mean_squared_error(y_val, SVM_pred)), r2_score(y_train, SVM_train),r2_score(y_val, SVM_pred), param])

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when 

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when 

In [9]:
RF_pred , RF_train, param = RF(X_train, y_train, X_val)
all_result.append([output, "Random Forest Regression", encoding,holder,np.sqrt(metrics.mean_squared_error(y_train, RF_train)), np.sqrt(metrics.mean_squared_error(y_val, RF_pred)), r2_score(y_train, RF_train),r2_score(y_val, RF_pred),str(param)])

In [10]:
XG_pred, XG_train, param = XGBR(X_train, y_train, X_val)
all_result.append([output,"XGB Regression",encoding,holder, np.sqrt(metrics.mean_squared_error(y_train, XG_train)), np.sqrt(metrics.mean_squared_error(y_val, XG_pred)), r2_score(y_train, XG_train),r2_score(y_val, XG_pred),str(param)]) 

In [11]:
NNR_pred, NNR_train, param = NNR(X_train, y_train, X_val)
all_result.append([output,"Neural Network Regression",encoding,holder, np.sqrt(metrics.mean_squared_error(y_train, NNR_train)), np.sqrt(metrics.mean_squared_error(y_val, NNR_pred)), r2_score(y_train, NNR_train),r2_score(y_val, NNR_pred),str(param)]) 


/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_p

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_p

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expecte

...

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expecte

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_p

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_p

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expecte

In [12]:
EN_pred , EN_train, param = EN(X_train, y_train, X_val)
all_result.append([output,"Elastic Net Regression",encoding,holder, np.sqrt(metrics.mean_squared_error(y_train, EN_train)), np.sqrt(metrics.mean_squared_error(y_val, EN_pred)), r2_score(y_train, EN_train),r2_score(y_val, EN_pred),str(param)])

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.413e+06, tolerance: 3.557e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.613e+06, tolerance: 3.948e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the 

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.160e+06, tolerance: 3.404e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.082e+06, tolerance: 4.144e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.942e+05, tolerance: 3.557e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.200e+06, tolerance: 3.948e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.216e+06, tolerance: 3.404e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.144e+06, tolerance: 4.144e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.942e+05, tolerance: 3.557e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.200e+06, tolerance: 3.948e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

...

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.193e+05, tolerance: 2.492e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.940e+05, tolerance: 3.557e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.194e+05, tolerance: 2.492e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.938e+05, tolerance: 3.557e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.937e+05, tolerance: 3.557e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.124e+06, tolerance: 3.948e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.916e+05, tolerance: 3.557e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.120e+06, tolerance: 3.948e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.110e+06, tolerance: 3.948e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.135e+06, tolerance: 3.404e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

In [13]:
dfResults = pd.DataFrame(all_result, columns=['Output' , 'Algorithm','Encoding Method' ,'Code', "RMSE Training", 'RMSE Val',"R^2 train","R^2 Val","Best_Parameter"])

In [14]:
dfResults

,Output,Algorithm,Encoding Method,Code,RMSE Training,RMSE Val,R^2 train,R^2 Val,Best_Parameter
0,pNP-Glc Ki (mM),Decision Tree Regression,BLOSUM62,NaN,136.046805,343.166895,0.681212,-0.235992,"{'max_features': 6, 'random_state': 20}"
1,pNP-Glc Ki (mM),Support Vector Machine Regression,BLOSUM62,NaN,269.588603,375.332375,-0.251783,-0.478553,"{'C': 10, 'coef0': 0.01, 'degree': 3}"
2,pNP-Glc Ki (mM),Random Forest Regression,BLOSUM62,NaN,187.661461,321.272909,0.393437,-0.083310,"{'max_depth': 100, 'min_samples_leaf': 4, 'min..."
3,pNP-Glc Ki (mM),XGB Regression,BLOSUM62,NaN,137.543315,330.411442,0.674160,-0.145816,"{'colsample_bytree': 1.0, 'gamma': 0.5, 'max_d..."
4,pNP-Glc Ki (mM),Neural Network Regression,BLOSUM62,NaN,238.061796,317.283739,0.023875,-0.056575,"{'activation': 'tanh', 'alpha': 0.0005, 'hidde..."
5,pNP-Glc Ki (mM),Elastic Net Regression,BLOSUM62,NaN,217.296688,316.433151,0.186735,-0.050918,"{'alpha': 100, 'l1_ratio': 0.9, 'max_iter': 1}"


In [15]:
name = encoding + "_hyperparameter_tune_ki.xlsx"
dfResults.to_excel(name)  